# Train NER with SpaCy

See [train_ner.py](https://github.com/explosion/spaCy/blob/master/examples/training/train_ner.py).

In [10]:
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
import random

# training data
TRAIN_DATA = [
    ("Who is Shaka Khan?", {"entities": [(7, 17, "PERSON")]}),
    ("I like London and Berlin.", {"entities": [(7, 13, "LOC"), (18, 24, "LOC")]}),
]

In [11]:
def get_nlp(model=None):
  if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
  else:
      nlp = spacy.blank("en")  # create blank Language class
      print("Created blank 'en' model")
  return nlp

def train(model=None, output_dir=None, n_iter=10):
    """Load the model, set up the pipeline and train the entity            recognizer."""
    nlp = get_nlp(model)

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
        
    def add_labels(ner):
        # add labels
        for _, annotations in TRAIN_DATA:
            for ent in annotations.get("entities"):
                ner.add_label(ent[2])

    add_labels(ner)

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)
    return nlp

def test(nlp):
    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

def save(nlp, output_dir=None):
      output_dir = Path(output_dir)
      if not output_dir.exists():
          output_dir.mkdir()
      nlp.to_disk(output_dir)
      print("Saved model to", output_dir)
        
def load(output_dir):
      # test the saved model
      print("Loading from", output_dir)
      nlp = spacy.load(output_dir)
      for text, _ in TRAIN_DATA:
          doc = nlp(text)
          print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
          print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])


def main():
  nlp = train()
  test(nlp)
  save(nlp, output_dir="models")
  nlp2 = load(output_dir="models")
  # Expected output:
  # Entities [('Shaka Khan', 'PERSON')]
  # Tokens [('Who', '', 2), ('is', '', 2), ('Shaka', 'PERSON', 3),
  # ('Khan', 'PERSON', 1), ('?', '', 2)]
  # Entities [('London', 'LOC'), ('Berlin', 'LOC')]
  # Tokens [('I', '', 2), ('like', '', 2), ('London', 'LOC', 3),
  # ('and', '', 2), ('Berlin', 'LOC', 3), ('.', '', 2)]

In [8]:
main()

Created blank 'en' model
Losses {'ner': 9.777777671813965}
Losses {'ner': 9.541139602661133}
Losses {'ner': 9.168060302734375}
Losses {'ner': 8.870099544525146}
Losses {'ner': 8.51644766330719}
Losses {'ner': 8.410200238227844}
Losses {'ner': 7.853942275047302}
Losses {'ner': 7.871019959449768}
Losses {'ner': 7.178551375865936}
Losses {'ner': 6.534702003002167}
Entities []
Tokens [('Who', '', 2), ('is', '', 2), ('Shaka', '', 2), ('Khan', '', 2), ('?', '', 2)]
Entities []
Tokens [('I', '', 2), ('like', '', 2), ('London', '', 2), ('and', '', 2), ('Berlin', '', 2), ('.', '', 2)]
Saved model to models
Loading from models
Entities []
Tokens [('Who', '', 2), ('is', '', 2), ('Shaka', '', 2), ('Khan', '', 2), ('?', '', 2)]
Entities []
Tokens [('I', '', 2), ('like', '', 2), ('London', '', 2), ('and', '', 2), ('Berlin', '', 2), ('.', '', 2)]
